In [97]:
from utilities.utils import crawl_directory

In [98]:
import os
import sklearn as sk
import numpy as np

In [271]:
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [308]:
from sklearn.metrics import accuracy_score, precision_score, recall_score ,f1_score, roc_auc_score

In [111]:
labels_path = "annotations/label"
video_features = "DATA/visual_features"
audio_features = "DATA/audio_features"

In [112]:
print("Crawling data paths")

Crawling data paths


In [157]:
labels_tree = crawl_directory(labels_path)
videos_tree = crawl_directory(video_features)
audio_tree = crawl_directory(audio_features)

In [158]:
print("Total Number of features and annotations")

Total Number of features and annotations


In [159]:
print("Labels len: ", len(labels_tree))
print("Audio len: ", len(audio_tree))
print("Video len: ", len(videos_tree))

Labels len:  418
Audio len:  418
Video len:  418


In [160]:
print("Sorting both lists with paths, in order to be in parallel")

Sorting both lists with paths, in order to be in parallel


In [161]:
# Sorting files list in order to be in parallel

labels_tree.sort()
videos_tree.sort()
audio_tree.sort()

In [162]:
len(labels_tree) == len(videos_tree) == len(audio_tree)


True

In [163]:
def unique_files(tree: list):
    """
    Getting the unique file names from a list
    """
    sunolo = set()
    for i in tree:
        sunolo.add(i.split(os.sep)[-1].split('.')[0])
    return sunolo

In [233]:
def load_npys_to_matrics(labels: list, videos: list, audio: list) -> tuple:
    """
    Loading the numpy files. Visual and audio will be averaged every 5 and 10 rows respectively. 
    DISCLAIMER i keep the minimum number of samples between the same video file from label, video and audio features matrices.
    """
    print("Nunpy to Matrices have start")
    files_sizes = []
    labels_matrix = []
    visual_matrix = []
    audio_matrix = []
    if not len(labels) == len(videos) == len(audio):
        raise Exception("Labels, visual features and audio have not the same size")
    for idx in range(len(labels)):
        
      
        # load labels, visual and audio in temporary variables
        try:
            
            tmp_label = np.load(labels[idx])
            tmp_visual = np.load(videos[idx])
            tmp_audio = np.load(audio[idx]).transpose() # transposed to the same format of visual features (rows = samplles, columns = features)
        except ValueError:
            print(f'File in index {idx} with name {videos[idx]} Not loaded')
            continue
        
        # get min seconds from the same label, visual, audio np file 
        l_r = tmp_label.shape[0]
        v_r, v_c = tmp_visual.shape
        a_r, a_c = tmp_audio.shape
        
        v_r = v_r//5
        a_r = a_r//10
        min_seconds = min(l_r, v_r, a_r)

        files_sizes.append(min_seconds)
        
        labels_matrix.append(tmp_label[:min_seconds])
        # VISUAL
        # keep number of samples divisible with 5
        tmp_visual = tmp_visual[:min_seconds*5]
        # averaging visual every 5 (Because we have analyze video with .2 step)
        visual_matrix.append(tmp_visual.transpose().reshape(-1,5).mean(1).reshape(v_c,-1).transpose())
        
        # AUDIO
        # keep number of samples divisible with 10
        tmp_audio = tmp_audio[:min_seconds*10]
        # averaging audio every 10 (Because we have analyze video with .1 step)
        audio_matrix.append(tmp_audio.transpose().reshape(-1,10).mean(1).reshape(a_c,-1).transpose())
        
        
        del tmp_label
        del tmp_visual
        del tmp_audio
        
    return files_sizes, labels_matrix, visual_matrix, audio_matrix

In [267]:
def split(labels: list, videos: list, audio: list, split_size: float) -> tuple:
    if not len(labels) == len(videos) == len(audio):
        raise Exception("Labels, visual features and audio have not the same size")
    if split_size >= 1.0 or split_size <= 0.0:
        raise Exception("Split size is out of bound")
    trainining_size = int(split_size * len(labels))
    #first training, second test
    return np.hstack([label for label in labels[:trainining_size]]),np.vstack([video for video in videos[:trainining_size]]), np.vstack([audio for audio in audio[:trainining_size]]), \
           np.hstack([label for label in labels[trainining_size:]]), np.vstack([video for video in videos[trainining_size:]]), np.vstack([audio for audio in audio[trainining_size:]])
    

In [256]:
# auto den paizei 
# video_72 = np.load(videos_tree[72])

In [235]:
files_sizes, labels_matrix, visual_matrix, audio_matrix = load_npys_to_matrics(labels_tree, videos_tree, audio_tree)

Nunpy to Matrices have start
File in index 72 with name DATA/visual_features/downhill/Matt Jones - DOWNHILL BRAKELESS BMX IS DANGEROUS!!.mp4.npy Not loaded


In [268]:
train_labels, train_visual, train_audio, test_labels, test_visual, test_audio = split(labels_matrix, visual_matrix, audio_matrix, 0.8)

In [269]:
print("TRAIN SHAPES")
print(f'Labels Shape {train_labels.shape}')
print(f'Visual Shape {train_visual.shape}')
print(f'Audio Shape {train_audio.shape}')


TRAIN SHAPES
Labels Shape (165437,)
Visual Shape (165437, 88)
Audio Shape (165437, 68)


In [270]:
print("TEST SHAPES")
print(f'Labels Shape {test_labels.shape}')
print(f'Visual Shape {test_visual.shape}')
print(f'Audio Shape {test_audio.shape}')


TEST SHAPES
Labels Shape (39540,)
Visual Shape (39540, 88)
Audio Shape (39540, 68)


## Fisrt Try Every number grater than 1, will be 1. Making binary classification (dummy) 

In [279]:
zero_one_labels = train_labels.copy()

In [280]:
zero_one_labels[zero_one_labels>1] = 1


In [287]:
zero_one_labels.shape

(165437,)

In [292]:
reshaped = zero_one_labels.reshape(-1,1)

In [294]:
reshaped.shape

(165437, 1)

#### Logistic Regression

In [303]:
lR = LogisticRegression(solver='lbfgs', max_iter=10000)

In [305]:
%%time
lR.fit(train_visual, zero_one_labels)

CPU times: user 3min 26s, sys: 1min 35s, total: 5min 2s
Wall time: 38.6 s


LogisticRegression(max_iter=10000)

In [307]:
predicted = lR.predict(test_visual)

In [311]:
acc = accuracy_score(test_labels, predicted)

In [321]:
f1 = f1_score(test_labels, predicted, average='macro')

In [323]:
recall = recall_score(test_labels, predicted, average='macro')

In [325]:
precision = precision_score(test_labels, predicted, average='macro', zero_division=0)

In [326]:
print(f'Accuracy {acc}')
print(f'f1 {f1}')
print(f'Recall { recall}')
print(f'Precision {precision}')


Accuracy 0.4853313100657562
f1 0.14350974680376488
Recall 0.15378507378043757
Precision 0.13683928975566778
